# Maximisation couple moyen

Ce notebook propose d'optimiser le rotor d'une machine synchro-réluctante sans aimant à une paire de pôle en vue de maximiser le couple moyen.
Dans ce cas particulier, le couple moyen est directement relié à la différence de perméance magnétique du rotor dans l'axe direct d et l'axe en quadrature q.

Le problème étudié est donc la maximisation de cette différence de perméance via la différence de compliance magnétique. Il est possible de démontrer que ce problème est mal posé.

In [56]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import CSG2d, Circle, Rectangle
from copy import copy
import numpy as np
import matplotlib.pyplot as plt
from ngsolve.internal import visoptions 
visoptions.scalfunction = "Flux:0"

## 1) Définition du maillage

La première étape consiste à définir une géométrie (disque), ainsi que la finesse de maillage associée.

In [2]:
def generate_fitted_circle_mesh(N):
    geo = CSG2d()
    R=1
    x = R*2*(np.append(np.insert(np.arange(0.5,N+0.5),0,0),N)/N-0.5)
    
    circle1 = Circle( center=(0,0), radius=R, bc="left_up" ) * Rectangle( pmin=(-R,0), pmax=(0,R))
    circle2 = Circle( center=(0,0), radius=R, bc="left_bot" ) * Rectangle( pmin=(-R,-R), pmax=(0,0))
    circle3 = Circle( center=(0,0), radius=R, bc="right_bot" ) * Rectangle( pmin=(0,-R), pmax=(R,0))
    circle4 = Circle( center=(0,0), radius=R, bc="right_up" ) * Rectangle( pmin=(0,0), pmax=(R,R))
    
    materials = ["iron","air"]
    
    for i in range(len(x)-1):
        geo.Add(Rectangle( pmin=(x[i],-R), pmax=(x[i+1],R), mat = materials[i%2] ) * (circle1 + circle2 + circle3 +circle4))

    #m = geo.GenerateMesh(maxh=max([R/N,1/30])) # On doit fixer la taille du maillage sinon le volume change à cause des elts grossiers
    m = geo.GenerateMesh(maxh=1/N)
    return Mesh(m)

mesh = generate_fitted_circle_mesh(30)
Draw (mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2105-9-g5a835126f', 'mesh_dim': 2, 'order2d': 1, 'order3d': 1, 'dr…

BaseWebGuiScene

Tenseur de perméabilité anisotrope homogénéisé :

$$\mu^*(\theta,\rho) = R(\theta)^T \underbrace{\begin{bmatrix} \mu_{D}(\rho) & 0 \\ 0 & \mu_{Q}(\rho) \end{bmatrix}}_{M(\rho)} R(\theta) $$

Avec $\rho$ la fraction volumique ($\simeq$ densité) et $\theta\in[-\pi/2,\pi/2]$ l'orientation privilégiée ($\theta = 0 \Rightarrow$ fibre orientée selon $\vec{x}$). $R$ est une matrice de rotation :
$$ R(\theta) = \begin{bmatrix} \cos(\theta) & - \sin(\theta) \\ \sin(\theta) & \cos(\theta) \end{bmatrix}$$

On définit aussi la perméabilité dans l'axe D (parallèle au flux):

$$\mu_{D}(\rho) = \mu_0 [\rho (\mu_r-1) + 1 ]$$

Et la perméabilité dans l'axe Q (perpendiculaire au flux):

$$\mu_{Q}(\rho) = \frac{\mu_0\mu_r}{\mu_r(1-\rho) + 1}$$


In [66]:
mu0 = 4e-7 * np.pi
mur = 1000

varspace = L2(mesh, order=0)    
rho = GridFunction(varspace)
theta = GridFunction(varspace)

rho.Set(0.5)
theta.Set(0)

def R(th):
    return CoefficientFunction( ( (cos(th),-sin(th)), (sin(th), cos(th)) ), dims = (2,2) )

def tR(th):
    return CoefficientFunction( ( (cos(th),sin(th)), (-sin(th), cos(th)) ), dims = (2,2) )

def muD(rh): 
    return mu0*(rh*(mur-1)+1)

def muQ(rh):
    return mu0*mur/(mur*(1-rh)+1)

def M(rh):
    return CoefficientFunction( ( (muD(rh),0),(0,muQ(rh)) ), dims = (2,2) )

def mu_star(rh,th):
    return tR(th)*M(rh)*R(th)

def DrawMuStar(rh,th):
    Draw(rh*CoefficientFunction((cos(th),sin(th))),mesh,vectors = { "grid_size":40});

DrawMuStar(rho,theta)

WebGuiWidget(value={'ngsolve_version': '6.2.2105-9-g5a835126f', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'dr…

Le problème sous forme faible s'écrit :
$$ \int_\Omega \nabla \Phi \mu^* \nabla \phi = \int_{\partial \Omega} \Phi \beta \vec{x}|\vec{y}. \vec{n} $$

In [29]:
beta = 1;

fespace_H1 = H1(mesh, order=1)
fespace_H1.FreeDofs()[0] = False

def solvePb(rh,th):
    phi = fespace_H1.TrialFunction()
    psi = fespace_H1.TestFunction()
    K = BilinearForm(fespace_H1, symmetric=True)
    K += grad(psi)*(mu_star(rh,th)*grad(phi))*dx

    n= specialcf.normal(mesh.dim);
    
    l1 = LinearForm(fespace_H1)
    l1 += -psi* beta * sqrt(1-x*x)* ds(definedon=mesh.Boundaries("right_bot|left_bot"))
    l1 += psi*beta* sqrt(1-x*x)*ds(definedon=mesh.Boundaries("right_up|left_up"))

    l2 = LinearForm(fespace_H1)
    l2 += -psi*beta*sqrt(1-y*y)*ds(definedon=mesh.Boundaries("right_bot|right_up"))
    l2 += psi*beta*sqrt(1-y*y)*ds(definedon=mesh.Boundaries("left_bot|left_up"))
    
    K.Assemble()
    invK = K.mat.Inverse(inverse="sparsecholesky")
    l1.Assemble()
    l2.Assemble()
    
    phi1 = GridFunction(fespace_H1)  # solution
    phi1.vec.data =     invK * l1.vec
    phi2 = GridFunction(fespace_H1)  # solution
    phi2.vec.data =     invK * l2.vec
    
    return(phi1, phi2)

La compliance anisotrope s'écrit :
$$ J(\phi,\rho,\theta) = \frac{1}{2} \int_\Omega \nabla \phi .\mu^*(\rho,\theta) \nabla \phi $$

Elle admet des dérivées partielles non nulles par rapport à $\rho$ et $\theta$ :

$$ \langle \partial_{\rho} J, \Phi \rangle = \frac{1}{2} \int_\Omega \Phi . \left ( \nabla \phi .\frac{\partial\mu^*}{\partial \rho} \nabla \phi \right)$$
$$ \langle \partial_{\theta} J, \Phi \rangle = \frac{1}{2} \int_\Omega \Phi . \left ( \nabla \phi .\frac{\partial\mu^*}{\partial \theta} \nabla \phi \right)$$
$$ \langle \partial_{\phi} J, \Phi \rangle = \frac{1}{2} \int_\Omega \nabla \Phi . \left ( (\mu^* +\mu^{*T}) \nabla \phi\right) =  \int_\Omega \nabla \Phi \mu^* \nabla \phi $$

Le problème est donc auto-adjoint, mais il ne faut pas oublier les dérivées partielles par rapport à $\theta$ et $\rho$ dans l'expression de la dérivée de la compliance.

In [51]:
def ComputeGradient(phi1,phi2,rh,th):
    
    mu1 = mu(phi1,rho)
    mu2 = mu(phi2,rho)
    Lag = (grad(lb1)*mu1*grad(phi1) + grad(lb2)*mu2*grad(phi2))*dx # FV
    Lag += (grad(phi1)*mu1* grad(phi1) -  grad(phi2)*mu2* grad(phi2))/2*dx # fonction objectif
    rho_test = rho.space.TestFunction()
    temp = LinearForm(rho.space)
    temp += Lag.Diff(rho,rho_test)
    temp.Assemble()
    r_temp = GridFunction(rho.space)
    r_temp.vec.data = temp.vec
    

WebGuiWidget(value={'ngsolve_version': '6.2.2105-9-g5a835126f', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'dr…

BaseWebGuiScene

In [ ]:
#Draw (phi2, mesh);

## 3) Optimisation

Le problème est auto-adjoint. Ainsi, l'adjoint s'écrit $\psi = -\phi$ et l'identification $\mathcal{L}^2$ du gradient par rapport à $\mu$ s'écrit:
$$\partial_\mu \mathcal{L} = -\int_\Omega |\nabla \phi|^2 $$

In [ ]:
def gradient(phi1,phi2):
    return(-grad(phi1)*grad(phi1) + grad(phi2)*grad(phi2))

def objective(phi1,phi2,mu):
    return(Integrate(grad(phi1)*mu* grad(phi1) -  grad(phi2)*mu* grad(phi2), mesh))


In [ ]:
mur = np.logspace(0.1,3,20)
Nmesh = [10, 20, 40]
resultMU= []
resultJ = []

for k in range(len(Nmesh)):
    lastMU = []
    lastJ = []
    N = Nmesh[k]
    mesh = generate_fitted_circle_mesh(N)

    for j in range(len(mur)):
        fespace_mu = L2(mesh, order=0)    
        mu = GridFunction(fespace_mu)
        s = randint(3, size=len(mu.vec[:].FV()))-1
        #mu.vec[:]= 4e-7*3.14*(2+s[:]) # situation initiale aléatoire
        mu.vec[:]= 4e-7*3.14*1.5 # situation initiale uniforme
        J=[]
        mu0 = 4e-7*3.14
        mu_max = mur[j]* mu0
        step = (mu_max-mu0)/3
        g = GridFunction(fespace_mu)
        MU=[]
        for i in range(1000):
            phi1, phi2 = solvePb(mu,mesh)
            g.Set(gradient(phi1,phi2))
            g.vec[:]= np.sign(g.vec[:].FV().NumPy())
            J.append(objective(phi1,phi2,mu))
            MU.append(copy(mu))
        
            if i>0 and J[-1]< J[-2]:
                step = step*1.2
            elif i>0:
                step = step/2
        
            mu.Set(MU[i] - g*step)
            mu.vec[:].FV().NumPy()[mu.vec[:].FV().NumPy()<mu0]=mu0
            mu.vec[:].FV().NumPy()[mu.vec[:].FV().NumPy()>mu_max]=mu_max
            if np.isnan(np.sum(mu.vec[:].FV().NumPy())):
                break
            if step/(mu_max-mu0) < 1e-4:
                break
        lastMU.append(copy(MU[-2]))
        lastJ.append(copy(J[-2]))
        print(f'{lastJ[-1]} - mu_r = {mur[j]} - mesh = 1/{N}')
    resultJ.append(copy(lastJ))
    resultMU.append(copy(lastMU))

In [ ]:
i = 15
Draw(lastMU[-1],mesh,min=mu0,max=mur[-1]*mu0)

In [ ]:
#plt.plot(J)

In [ ]:
theta_opt = (1-np.sqrt(mur))/(1-mur);
Jopt = pi/mu0 * ((theta_opt/mur + (1-theta_opt)) - 1/(theta_opt*mur+ (1-theta_opt)))

plt.semilogx(mur,-np.array(lastJ),mur,Jopt)
plt.grid()
plt.xlabel('mu_r')
plt.ylabel('Fonction objectif (J)')
plt.legend(['obtenu','maximum théorique'])

In [ ]:
mur[-1]